In [1]:
!pip install --upgrade pyswarm
!pip install pymc3
!pip install --upgrade pactools


  Created wheel for pyswarm: filename=pyswarm-0.6-cp36-none-any.whl size=4481 sha256=6a30d2a0d5014b341443eeb8a83abb6b18703931bb86b1a831286060718b6353
  Stored in directory: /root/.cache/pip/wheels/37/c5/f6/b33b9ac00040cb95c1f00af982a4197334a672d6de43f4699f
Successfully built pyswarm
     |████████████████████████████████| 92kB 2.6MB/s 


In [2]:
from sklearn.model_selection import train_test_split
from pyswarm import pso
from os import path
import os
import requests
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy
import sys
import numpy as np
from numpy import loadtxt
from numpy import array
from numpy.random import choice
import pandas as pd
import time
import random
import statistics
import pandas
import math
import csv
import random
import logging
from pymc3 import *
import pymc3 as pm
from functools import reduce
from operator import add
from tqdm import tqdm
import geopy.distance
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from theano import shared
from sklearn import preprocessing
print('Running on PyMC3 v{}'.format(pm.__version__))
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow
from tensorflow.keras import datasets, layers, models
from keras.utils import np_utils
from keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.ticker import MaxNLocator

#TESNORFOW
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models

#KERAS LIBRARIES
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout , Flatten,BatchNormalization,Conv2D,MaxPooling2D, Activation,LSTM,Embedding,Input,GlobalAveragePooling2D
from keras.regularizers import l1, l2, l1_l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend 
from keras.utils import np_utils
from keras.utils import to_categorical



/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Running on PyMC3 v3.7


In [3]:
train1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train.npy',allow_pickle=True)
train_labels1 = np.load('/content/drive/My Drive/NumpyArrayCovidx/train_labels.npy',allow_pickle=True)

In [4]:
train2,test1, train_labels2,test_labels1 = train_test_split(train1, train_labels1, test_size=0.2, random_state=42)

In [5]:
train=train2/225.0
train_labels = pd.get_dummies(train_labels2)

test=test1/225.0
test_labels = pd.get_dummies(test_labels1)


In [6]:
IMG_SHAPE1=(64,64,3) # θα το προσαρμόσουμε

vgg19 = keras.applications.vgg19.VGG19(input_shape=IMG_SHAPE1,
                                               include_top=False,
                                               weights='imagenet')

80142336/80134624 [==============================] - 2s 0us/step


In [7]:
#fine_tuning,flatvsglobalpool,dropout,dense_layers
lb=[0,0,0,0]
ub=[19,1,0.5,4]

In [8]:
def create_model_dense_vgg(x): 
  tempvg=vgg19
  for layer in tempvg.layers[:(-1)*int(round(x[0]))]:
    layer.trainable = False
  
  #vgg19.trainable=False
  model = tf.keras.Sequential()
  model.add(tempvg)
  if (int(round(x[1]))):
    model.add(keras.layers.Flatten())
  else:
    model.add(keras.layers.GlobalAveragePooling2D())
  if (int(round(x[3]))==4):
    layers=[256,128,64,32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  elif (int(round(x[3]))==3):
    layers=[128,64,32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  elif (int(round(x[3]))==2):
    layers=[64,32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  elif (int(round(x[3]))==1):
    layers=[32]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))
  else:
    layers=[]
    for i in range(len(layers)):
          model.add(keras.layers.Dense(layers[i], activation='relu'))
          model.add(keras.layers.Dropout((x[2])))

  for layer in model.layers:
    print(layer, layer.trainable)
  model.add(keras.layers.Dense(3,activation="softmax"))
  model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=["accuracy"])
  return model

In [9]:
EarlyStopper = EarlyStopping(patience=4, monitor='val_loss', mode='min')

In [11]:
mode=create_model_dense_vgg([5,1,0.1,3])
mode.summary()

<tensorflow.python.keras.engine.functional.Functional object at 0x7f7e15c087f0> True
<tensorflow.python.keras.layers.core.Flatten object at 0x7f7d944875c0> True
<tensorflow.python.keras.layers.core.Dense object at 0x7f7d944876d8> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7f7d94431e10> True
<tensorflow.python.keras.layers.core.Dense object at 0x7f7d94431f98> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7f7d94443ef0> True
<tensorflow.python.keras.layers.core.Dense object at 0x7f7d9444c2e8> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7f7d94487320> True
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 2, 2, 512)         20024384  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_____________________________________

In [12]:
def apple(x):
  model = create_model_dense_vgg(x)
  model.fit(train, train_labels, epochs=20, batch_size=1000, verbose=1,validation_data=(test, test_labels),callbacks=[EarlyStopper])
  loss, _ = model.evaluate(test, test_labels, verbose=1)
  return loss 

In [13]:
xopt, fopt = pso(apple, lb, ub, swarmsize=10, omega=0.5, phip=0.5, phig=1.0, maxiter=30, minstep=1)
print ("Best position"+str(xopt))
print ("Loss:" + str(fopt))

<tensorflow.python.keras.engine.functional.Functional object at 0x7f7e15c087f0> True
<tensorflow.python.keras.layers.pooling.GlobalAveragePooling2D object at 0x7f7d943f8a58> True
<tensorflow.python.keras.layers.core.Dense object at 0x7f7e00075c18> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7f7d943b0940> True
<tensorflow.python.keras.layers.core.Dense object at 0x7f7d943b0fd0> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7f7d943c5fd0> True
Epoch 1/20
17/17 [==============================] - 5s 307ms/step - loss: 1.3960 - accuracy: 0.4186 - val_loss: 0.9803 - val_accuracy: 0.4968
Epoch 2/20
17/17 [==============================] - 4s 241ms/step - loss: 0.8726 - accuracy: 0.5859 - val_loss: 0.7308 - val_accuracy: 0.6523
Epoch 3/20
17/17 [==============================] - 4s 241ms/step - loss: 0.6582 - accuracy: 0.7166 - val_loss: 0.4722 - val_accuracy: 0.8115
Epoch 4/20
17/17 [==============================] - 4s 241ms/step - loss: 0.4535 - accuracy: 0.

In [14]:
model=create_model_dense_vgg([3,0,0.06,0])

<tensorflow.python.keras.engine.functional.Functional object at 0x7f7e15c087f0> True
<tensorflow.python.keras.layers.pooling.GlobalAveragePooling2D object at 0x7f7d943f8e80> True


In [16]:
model.fit(train, train_labels, epochs=30, batch_size=1000, verbose=1,validation_data=(test, test_labels))

Epoch 1/30
17/17 [==============================] - 4s 216ms/step - loss: 0.0847 - accuracy: 0.9785 - val_loss: 0.2578 - val_accuracy: 0.9183
Epoch 2/30
17/17 [==============================] - 4s 211ms/step - loss: 0.0819 - accuracy: 0.9788 - val_loss: 0.2607 - val_accuracy: 0.9181
Epoch 3/30
17/17 [==============================] - 4s 211ms/step - loss: 0.0796 - accuracy: 0.9790 - val_loss: 0.2636 - val_accuracy: 0.9186
Epoch 4/30
17/17 [==============================] - 4s 212ms/step - loss: 0.0776 - accuracy: 0.9793 - val_loss: 0.2666 - val_accuracy: 0.9183
Epoch 5/30
17/17 [==============================] - 4s 212ms/step - loss: 0.0759 - accuracy: 0.9796 - val_loss: 0.2694 - val_accuracy: 0.9186
Epoch 6/30
17/17 [==============================] - 4s 212ms/step - loss: 0.0745 - accuracy: 0.9798 - val_loss: 0.2722 - val_accuracy: 0.9186
Epoch 7/30
17/17 [==============================] - 4s 212ms/step - loss: 0.0732 - accuracy: 0.9798 - val_loss: 0.2749 - val_accuracy: 0.9186
Epoch 

In [17]:
model1=create_model_dense_vgg([15,0,0.06,3])
model1.fit(train, train_labels, epochs=60, batch_size=1000, verbose=1,validation_data=(test, test_labels))

<tensorflow.python.keras.engine.functional.Functional object at 0x7f7e15c087f0> True
<tensorflow.python.keras.layers.pooling.GlobalAveragePooling2D object at 0x7f798d84e470> True
<tensorflow.python.keras.layers.core.Dense object at 0x7f798d84e7f0> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7f798d7f5ac8> True
<tensorflow.python.keras.layers.core.Dense object at 0x7f798d84ec18> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7f798d810278> True
<tensorflow.python.keras.layers.core.Dense object at 0x7f798d810748> True
<tensorflow.python.keras.layers.core.Dropout object at 0x7f798d8224a8> True
Epoch 1/60
17/17 [==============================] - 4s 227ms/step - loss: 0.3767 - accuracy: 0.8954 - val_loss: 0.2897 - val_accuracy: 0.9166
Epoch 2/60
17/17 [==============================] - 4s 213ms/step - loss: 0.0768 - accuracy: 0.9773 - val_loss: 0.3787 - val_accuracy: 0.9183
Epoch 3/60
17/17 [==============================] - 4s 213ms/step - loss: 0.0700 - accu

KeyboardInterrupt: ignored